In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = TextLoader("../data/cafe_menu_data.txt", encoding='utf-8')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
splits = splitter.split_documents(docs)

embedding = OllamaEmbeddings(model="bge-m3")
db = FAISS.from_documents(splits, embedding)
db.save_local("./db/cafe_db")


In [8]:
from typing import List
from langchain_core.tools import tool
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

tavily = TavilySearchResults(max_results=3)
wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)

# cafe DB 불러오기
retriever = FAISS.load_local("./db/cafe_db", embedding, allow_dangerous_deserialization=True).as_retriever()

@tool
def db_search_cafe_func(query: str) -> List[str]:
    """카페 메뉴 DB에서 메뉴 정보를 검색합니다."""
    docs = retriever.invoke(query)
    return [doc.page_content for doc in docs]

@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다."""
    return str(tavily.invoke(query))

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 요약 정보를 검색합니다."""
    return str(wiki.run(query))


C:\Users\user\AppData\Local\Temp\ipykernel_3312\52309614.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults(max_results=3)


In [9]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")  # 또는 "gpt-3.5-turbo"
llm_with_tools = llm.bind_tools([db_search_cafe_func, tavily_search_func, wiki_summary])


In [12]:
from langchain_core.messages import HumanMessage, ToolMessage

# 사용자 질문 → 아메리카노 정보 요청
query = "아메리카노의 가격과 특징은 무엇인가요?"

# LLM에게 질의 → 어떤 도구 호출할지 판단
ai_msg = llm_with_tools.invoke(query)

# 도구 호출 정보 확인
if ai_msg.tool_calls:
    tool_call = ai_msg.tool_calls[0]
    print("도구 호출:", tool_call['name'])

    # 도구 실행
    if tool_call['name'] == 'db_search_cafe_func':
        tool_output = db_search_cafe_func.invoke(tool_call['args'])

        # 도구 결과를 ToolMessage로 래핑
        tool_msg = ToolMessage(
            content=str(tool_output),
            tool_call_id=tool_call['id'],
            name=tool_call['name']
        )

        # 최종 응답 재요청
        final_response = llm_with_tools.invoke(
            [HumanMessage(content=query), ai_msg, tool_msg]
        )

        print("최종 응답:")
        print(final_response.content)

else:
    # 도구 호출 없이 바로 응답
    print("도구 없이 직접 응답:", ai_msg.content)


도구 호출: db_search_cafe_func
최종 응답:
아메리카노는 ₩4,500에 판매되며, 주요 원료는 에스프레소와 뜨거운 물입니다. 진한 에스프레소에 뜨거운 물을 더해 만들어진 클래식한 블랙 커피로, 깔끔하고 깊은 풍미가 특징입니다. 추가로 설탕이나 시럽을 넣어 맞춤형으로 즐길 수 있습니다.
